In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

In [1]:
# Загружаем YOLOv8 модель
model = YOLO("yolov8/best.pt")
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Открываем видео
video_path = "../input_data/rat_test.mp4"
cap = cv2.VideoCapture(video_path)

# Получаем ширину, высоту, FPS
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)


out = cv2.VideoWriter('../yolo-v8-main/runs/detect/predict2/rat_test.mp4', fourcc, fps, (frame_width, frame_height))

# Создаем матрицу тепловой карты
heatmap = np.zeros((frame_height, frame_width), dtype=np.float32)

# Записываем координаты с траекториями
trajectory_points = []

def apply_heatmap(heatmap):
    heatmap_normalized = cv2.normalize(heatmap, None, 0, 255, cv2.NORM_MINMAX)
    heatmap_colored = cv2.applyColorMap(heatmap_normalized.astype(np.uint8), cv2.COLORMAP_JET)
    return heatmap_colored

# Определяем цвет в зависимости от посещений
def get_color(visits):
    if visits == 1:
        return (0, 255, 0)  
    elif visits == 2:
        return (0, 255, 255)  
    else:
        return (0, 0, 255)  

# Работаем с видео
while cap.isOpened():
    success, frame = cap.read()

    if not success:
        break

    results = model(frame)

    for result in results:
        for box in result.boxes:
            # Получаем координаты крысы
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Рисуем прямоугольник вокруг крысы
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Получаем координаты центра прямоугольника
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)
            
            heatmap[center_y, center_x] += 1
            
            # Рисуем траекторию
            trajectory_points.append((center_x, center_y, heatmap[center_y, center_x]))

    # Рисуем кружки на точках перемещения
    for x, y, visits in trajectory_points:
        color = get_color(int(visits))
        radius = int(10 * np.sqrt(visits))
        cv2.circle(frame, (x, y), radius, color, -1)

    # Применяем сглаживание на тепловую карту
    heatmap_smoothed = cv2.GaussianBlur(heatmap, (15, 15), 0)

    colored_heatmap = apply_heatmap(heatmap_smoothed)

    overlay = cv2.addWeighted(frame, 0.6, colored_heatmap, 0.4, 0)

    out.write(overlay)

    # Display the frame (optional)
    # cv2.imshow('YOLOv8 Trajectory Heatmap', overlay)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

final_heatmap = apply_heatmap(heatmap_smoothed)
cv2.imwrite('../yolo-v8-main/runs/detect/predict2/final_heatmap.png', final_heatmap)


c:\Users\Lev\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

0: 352x416 1 rat, 189.2ms
Speed: 8.0ms preprocess, 189.2ms inference, 11.0ms postprocess per image at shape (1, 3, 352, 416)

0: 352x416 1 rat, 101.0ms
Speed: 2.0ms preprocess, 101.0ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 416)

0: 352x416 1 rat, 132.0ms
Speed: 3.0ms preprocess, 132.0ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 416)

0: 352x416 1 rat, 130.8ms
Speed: 3.4ms preprocess, 130.8ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 416)

0: 352x416 1 rat, 132.0ms
Speed: 3.0ms preprocess, 132.0ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 416)

0: 352x416 1 rat, 310.6ms
Speed: 4.0ms preprocess, 310.6ms inference, 1.0ms postprocess per image 

True